# FEXI notebook

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
from tqdm import tqdm
from scipy.special import erf
import scipy.stats

from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.core.modeling_framework import MultiCompartmentSphericalMeanModel
from dmipy.signal_models import sphere_models, cylinder_models, gaussian_models

from scipy.io import savemat

/Users/admin/opt/anaconda3/envs/project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### FEXI simulations
#### Simulated $s$ signal

In [28]:
def axr_sim(adc, sigma, axr, bf, be, tm):
   """Generate an output signal s based on known inputs for a given voxel
Inputs    - adc:      apparent diffusion coefficient [m2/s] single value
         - sigma:    filter efficiency single value
         - axr:      exchange rate [1/s] single value
         - bf:       filter block b-value [m2/s] 20x1
         - be:       encoding block b-value [m2/s] 20x1
         - tm:       mixing time [s] 20x1

Output: 	- s:        signal (sum of the magnetisations) single value
   Based off Elizabeth's code. 
   """

   tm[(bf == 0) & (tm == np.min(tm))] = np.inf
   #calculate ADC as fnc of mixing time
   adc_tm = adc * (1 - sigma* np.exp(-axr*tm))

   # compute signal
   s = np.exp(-adc_tm*be)
   return s 


Say we wanted to calculate s for a random distribution of adc values.
I think axr may also vary, but both are probs linked to each other. 

In [29]:
#adc=1
sigma=2
axr=3
bf = np.array([1e-6, 0.090, 1e-6, 0, 1e-6, 1.5, 1e-6, 2, 1e-6, 3,1e-6, 0, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
be = np.array([1e-6, 0.090, 1e-6, 1, 1e-6, 1.5, 1e-6, 2, 1e-6, 3,1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
tm = np.array([1e-6, 0.090, 1e-6, -1, 1e-6, 1.5, 1e-6, 2, 1e-6, 3,1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
nvox = 1000

#axrs = np.random.uniform(1,100,nvox) 
adcs = np.random.uniform(200,300,nvox) 

#axr_sim(adc,sigma,axr,bf,be,tm)


s = np.array([axr_sim(adc,sigma,axr,bf,be,tm) for adc in adcs]) 
